In [4]:
import argparse
import os
from subprocess import check_call
import sys
import json
import logging

In [10]:
def launch_training_job(pretrain_path, learning_rate):
    """Launch training of the model with a set of hyperparameters in parent_dir/job_name
    Args:
        parent_dir: (string) directory containing config, weights and log
        data_dir: (string) directory containing the dataset
        params: (dict) containing hyperparameters
    """
    # Create a new folder in parent_dir with unique_name "job_name"
    root_path = "/home/shared/workspace/"
    pretrain_name = pretrain_path.split('_')[0]
    model_name = pretrain_path.split('_')[1]
    model_width = pretrain_path.split('_')[2]
    print(pretrain_name)
    print(model_name)
    print(model_width)

    result_name = model_name + "_" + model_width  + "_" + str(50) + "_" + str(learning_rate)
    result_dir = os.path.join(root_path, 'human-activity-recognition', 'Efficient-3DCNNs', 'data', 'results', result_name)
    if not os.path.exists(result_dir):
        print(result_dir)
#         os.makedirs(result_dir)


    # Launch training with this config
    num_classes = 600

    if pretrain_name == "kinetics":
        num_classes = 600
    elif pretrain_name == "jester" :
        num_classes = 27
    print(num_classes)
        
    cmd = "python main.py --root_path /home/shared/workspace/ \
        --video_path Resnet3D/3D-ResNets-PyTorch/data/jpg \
        --annotation_path Resnet3D/3D-ResNets-PyTorch/data/ntu_01.json \
        --result_path {result_dir} \
        --pretrain_path Resnet3D/3D-ResNets-PyTorch/data/models/{pretrain_path}_RGB_16_best.pth \
        --dataset ucf101 \
        --n_classes {num_classes} \
        --n_finetune_classes 9 \
        --ft_portion last_layer \
        --model {model_name} \
        --groups 3 \
        --width_mult 1.0 \
        --train_crop random \
        --learning_rate {learning_rate} \
        --sample_duration 16 \
        --downsample 1 \
        --n_threads 16 \
        --checkpoint 1 \
        --n_val_samples 1 \
        --batch_size 64 \
        --n_epochs 1".format(result_dir = result_dir, pretrain_path = pretrain_path, num_classes = num_classes,
                             model_name = model_name, learning_rate = learning_rate)
    print(cmd)
#     check_call(cmd, shell=True)

In [13]:
pretrain_path = ['kinetics_mobilenetv2_1.0x', 'kinetics_shufflenetv2_1.0x', 'kinetics_resnet_101', 'kinetics_resnext_101', 'kinetics_resnet_50', 'kinetics_resnet_18']

for p in pretrain_path :
    launch_training_job(p, 0.001)

kinetics
mobilenetv2
1.0x
/home/shared/workspace/human-activity-recognition/Efficient-3DCNNs/data/results/mobilenetv2_1.0x_50_0.001
600
python main.py --root_path /home/shared/workspace/         --video_path Resnet3D/3D-ResNets-PyTorch/data/jpg         --annotation_path Resnet3D/3D-ResNets-PyTorch/data/ntu_01.json         --result_path /home/shared/workspace/human-activity-recognition/Efficient-3DCNNs/data/results/mobilenetv2_1.0x_50_0.001         --pretrain_path Resnet3D/3D-ResNets-PyTorch/data/models/kinetics_mobilenetv2_1.0x_RGB_16_best.pth         --dataset ucf101         --n_classes 600         --n_finetune_classes 9         --ft_portion last_layer         --model mobilenetv2         --groups 3         --width_mult 1.0         --train_crop random         --learning_rate 0.001         --sample_duration 16         --downsample 1         --n_threads 16         --checkpoint 1         --n_val_samples 1         --batch_size 64         --n_epochs 1
kinetics
shufflenetv2
1.0x
/home/shared

In [19]:
def launch_inference_job(pretrain_path, model, learning_rate):
    """Launch training of the model with a set of hyperparameters in parent_dir/job_name
    Args:
        parent_dir: (string) directory containing config, weights and log
        data_dir: (string) directory containing the dataset
        params: (dict) containing hyperparameters
    """
    # Create a new folder in parent_dir with unique_name "job_name"
    root_path = "/home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data/"
    
    result_name = model + "_" + pretrain_path.split('.')[0] + "_" + str(learning_rate)
    result_dir = os.path.join(root_path, 'results', result_name)
    if not os.path.exists(result_dir):
        print("result_dir does not exist")

#     # Write parameters in json file
#     json_path = os.path.join(model_dir, 'params.json')
#     params.save(json_path)

    # Launch training with this config
    model_name = pretrain_path.split('.')[0]
    model_depth = model_name.split('_')[0].split('d')[-1]
    pretrain_dataset = model_name.split('_')[1]

    if pretrain_dataset == 'K':
        n_pretrain_classes = 700
    elif pretrain_dataset == 'KM':
        n_pretrain_classes = 1039
    elif pretrain_dataset == 'KMS':
        n_pretrain_classes = 1139
    elif pretrain_dataset == 'M':
        n_pretrain_classes = 339
    elif pretrain_dataset == 'S':
        n_pretrain_classes = 100
    else :
        n_pretrain_classes = 0
        
    cmd = "python main.py --root_path {root_path} \
    --video_path jpg \
    --annotation_path ntu_01.json \
    --result_path {result_dir} \
    --dataset ucf101 \
    --n_classes 9 \
    --model_depth {model_depth} \
    --n_threads 4 \
    --no_train \
    --no_val \
    --inference \
    --output_topk 3\
    --inference_batch_size 1".format(root_path = root_path, result_dir = result_dir, model_depth = model_depth)
    print(cmd)
    check_call(cmd, shell=True)

In [21]:
launch_inference_job("r3d50_KMS_200ep.pth", "resnet", 1e-4)

python main.py --root_path /home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data/     --video_path jpg     --annotation_path ntu_01.json     --result_path /home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data/results/resnet_r3d50_KMS_200ep_0.0001     --dataset ucf101     --n_classes 9     --model_depth 50     --n_threads 4     --no_train     --no_val     --inference     --output_topk 5    --inference_batch_size 1


In [26]:
def launch_topclass_job(pretrain_path, model, learning_rate):
    """Launch training of the model with a set of hyperparameters in parent_dir/job_name
    Args:
        parent_dir: (string) directory containing config, weights and log
        data_dir: (string) directory containing the dataset
        params: (dict) containing hyperparameters
    """
    # Create a new folder in parent_dir with unique_name "job_name"
    root_path = "/home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data/"
    
    result_name = model + "_" + pretrain_path.split('.')[0] + "_" + str(learning_rate)
    result_dir = os.path.join(root_path, 'results', result_name)
    if not os.path.exists(result_dir):
        print("result_dir does not exist")
        
    cmd = "python -m util_scripts.eval_accuracy \
    {root_path}/ntu_01.json \
    {result_dir}/val.json \
    --subset validation -k 1 --ignore".format(root_path = root_path, result_dir = result_dir)
    print(cmd)
    check_call(cmd, shell=True)

In [28]:
launch_topclass_job("r3d50_KMS_200ep.pth", "resnet", 1e-4)

python -m util_scripts.eval_accuracy     /home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data//ntu_01.json     /home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/data/results/resnet_r3d50_KMS_200ep_0.0001/val.json     --subset validation -k 1 --ignore
